<a href="https://colab.research.google.com/github/GokuKrish24/GokuKrish24/blob/main/Wastesegregation_using_HOG_with_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gokul240503","key":"b2512c14b0c0eb5e3147b60d1a23e6d7"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                              title                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------  ----------------------------------  -----  -------------------  -------------  ---------  ---------------  
zain280/car-dataset                              Car Dataset                           1MB  2024-03-02 15:14:49           1541         24  1.0              
syedanwarafridi/vehicle-sales-data               Vehicle Sales Data                   19MB  2024-02-21 20:16:17           7784        135  1.0              
mahad049/job-placement-dataset                   Job placement dataset                 6KB  2024-03-03 08:18:12            896         24  0.9411765        
yaminh/smartphone-sale-dataset                   Smartphones Sales Dataset            65KB  2024-03-03 16:48:27           1004         22  0.88235295       
tarunrm09/climate-change-indicators              Climate c

In [ ]:
! kaggle datasets download aashidutt3/waste-segregation-image-dataset
!unzip waste-segregation-image-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/train/biodegradable/leaf_waste/16613329-813d-4d3d-9f44-e52c0ad72bb0___RS_Late.B 4980.JPG  
  inflating: Dataset/train/biodegradable/leaf_waste/16984239-2e67-407e-b600-4a3666500534___RS_Late.B 5182.JPG  
  inflating: Dataset/train/biodegradable/leaf_waste/1745208a-82b2-4a82-9791-d812cb016850___GHLB2 Leaf 95.JPG  
  inflating: Dataset/train/biodegradable/leaf_waste/18568211-18d7-453f-84a4-d22e2088ec33___RS_Late.B 5239.JPG  
  inflating: Dataset/train/biodegradable/leaf_waste/1909260c-1e08-4a8f-a702-60549ada08b2___RS_Late.B 6671.JPG  
  inflating: Dataset/train/biodegradable/leaf_waste/192c9d29-4a86-457b-a779-1dd36f225a5b___RS_Early.B 8012.JPG  
  inflating: Dataset/train/biodegradable/leaf_waste/1a017b74-a549-433b-979b-3e3ace67b50d___GCREC_Bact.Sp 5715.JPG  
  inflating: Dataset/train/biodegradable/leaf_waste/1a067cc9-cf70-41a5-a51c-aa0f6b8a936d___UF.GRC_BS_Lab Leaf 0327.JPG  
  inflating: Dataset/train/biodegradable

In [ ]:
train_dir = '/content/Dataset/train'
val_dir = '/content/Dataset/val'

import os
import pandas as pd

def create_csv(dataset_dir, csv_filename):
    image_paths = []
    labels = []

    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png'):
                image_path = os.path.join(root, file)
                image_paths.append(image_path)
                label = os.path.basename(root)
                labels.append(label)

    data = pd.DataFrame({'image_path': image_paths, 'label': labels})
    data.to_csv(csv_filename, index=False)

create_csv(train_dir, 'train_dataset.csv')
create_csv(val_dir, 'val_dataset.csv')

In [ ]:
train_data = pd.read_csv('train_dataset.csv')
val_data = pd.read_csv('val_dataset.csv')

import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (128, 128))  # width=64, height=128
    # Convert image to grayscale
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Extract HOG features from the grayscale image
    features, hog_image = hog(gray_image, orientations=9, pixels_per_cell=(16, 16),
                              cells_per_block=(8, 8), visualize=True, block_norm='L2-Hys',transform_sqrt=True)
    return features

# Load and preprocess data
def load_and_preprocess_data(data):
    # Initialize lists to store features and labels
    features = []
    labels = []

    # Iterate over each row in the DataFrame
    for index, row in data.iterrows():
        # Preprocess image and extract HOG features
        image_features = preprocess_image(row['image_path'])
        # Append features to the list
        features.append(image_features)
        # Append label to the list
        labels.append(row['label'])

    # Convert lists to numpy arrays
    features = np.array(features)
    labels = np.array(labels)

    return features, labels

# Load data
train_features, train_labels = load_and_preprocess_data(train_data)
val_features, val_labels = load_and_preprocess_data(val_data)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
# Initialize SVM classifier
svm_classifier = LinearSVC()

# Train the classifier
svm_classifier.fit(X_train, y_train)

LinearSVC()

In [ ]:
from sklearn.metrics import accuracy_score

# Make predictions on the test/validation data
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8307459295721318
